In [ ]:
!nvidia-smi

Wed Dec  1 14:55:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import os
import keras.metrics
import matplotlib.pyplot as plt
from imutils import paths
from sklearn.model_selection import train_test_split

In [3]:
test_dir =  '/content/drive/MyDrive/Major Project/Datasets/Phase 1-Fruit Detection/fruits-360/Training/'

In [4]:
fruits = []
fruits_image = []
for i in os.listdir(test_dir):
    for image_filename in os.listdir(test_dir + i):
        fruits.append(i) # name of the fruit 
        fruits_image.append(i + '/' + image_filename)

In [5]:
test_fruits = pd.DataFrame(fruits, columns=["Fruits"])
test_fruits["Fruits Image"] = fruits_image

In [6]:
test_fruits

,Fruits,Fruits Image
0,S1,S1/r_115_100.jpg
1,S1,S1/168_100.jpg
2,S1,S1/158_100.jpg
3,S1,S1/211_100.jpg
4,S1,S1/r_182_100.jpg
...,...,...
66097,S131,S131/17_100.jpg
66098,S131,S131/18_100.jpg
66099,S131,S131/19_100.jpg
66100,S131,S131/20_100.jpg


In [7]:
image_paths=[]
data_labels=[]

In [8]:
for i in range(len(test_fruits)):
  path=test_fruits['Fruits Image'][i]
  path=test_dir+path
  label=test_fruits['Fruits'][i]
  image_paths.append(path)
  data_labels.append(label)

In [10]:
# Training and Validation split
from sklearn.model_selection import train_test_split
(train_data_path_list, image_paths, train_data_class_list, data_labels) = train_test_split(image_paths,
                                                                                                        data_labels,
                                                                                                        test_size = 0.9,
                                                                                                        stratify = data_labels,
                                                                                                        random_state = 101)

In [11]:
df_test = pd.DataFrame({"Path":image_paths,"Class":data_labels})

In [12]:
df_test

,Path,Class
0,/content/drive/MyDrive/Major Project/Datasets/...,S83
1,/content/drive/MyDrive/Major Project/Datasets/...,S15
2,/content/drive/MyDrive/Major Project/Datasets/...,S38
3,/content/drive/MyDrive/Major Project/Datasets/...,S105
4,/content/drive/MyDrive/Major Project/Datasets/...,S101
...,...,...
59487,/content/drive/MyDrive/Major Project/Datasets/...,S10
59488,/content/drive/MyDrive/Major Project/Datasets/...,S105
59489,/content/drive/MyDrive/Major Project/Datasets/...,S126
59490,/content/drive/MyDrive/Major Project/Datasets/...,S55


In [18]:
from keras.models import load_model

trained_model = load_model("/content/drive/MyDrive/Major Project/Training Files/Dataset1/Transfer Learning/Ratio 5/Models/vgg16-ep15.h5")

In [19]:
trained_model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 100, 100, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 100, 100, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 100, 100, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 50, 50, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 50, 50, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 50, 50, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 25, 25, 128)       0     

In [20]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [21]:
ts_data = ImageDataGenerator()
testing_df = ts_data.flow_from_dataframe(df_test, x_col = "Path",y_col = "Class", batch_size=1, target_size=(100, 100))

Found 59492 validated image filenames belonging to 131 classes.


In [22]:
trained_model.evaluate(testing_df)

59492/59492 [==============================] - 14030s 236ms/step - loss: 1.0155 - accuracy: 0.7747 - precision: 0.9626 - recall: 0.6556 - auc: 0.9848 - tp: 39004.0000 - fp: 1514.0000 - tn: 7732446.0000 - fn: 20488.0000


[1.015470027923584,
 0.7746924161911011,
 0.9626339077949524,
 0.6556175351142883,
 0.9847994446754456,
 39004.0,
 1514.0,
 7732446.0,
 20488.0]